## Library Imports

In [1]:
%matplotlib inline
%gui asyncio


# PI Camera Libs
import picamera
from picamera.array import PiRGBArray
from picamera import PiCamera

# Servo Control libs
from adafruit_servokit import ServoKit

# Opnen CV libs (For OCR)
from PIL import Image
import cv2
import numpy
import matplotlib.pyplot as plt
import pytesseract

# Google cloud visison related
from google.oauth2 import service_account
from google.cloud import vision

# import the necessary packages
# from imutils.perspective import four_point_transform
# from imutils import contours
import imutils
import threading
import asyncio
import re
import io
from time import sleep
import datetime as dt
import timeit

# Confirm opencv import were done correctly
print ("OPEN_CV VER:\t", cv2.__version__)
print ("NUMPY VER:\t", numpy.__version__)
print ("TESSERECT VER:\t", pytesseract.__version__)

# some handy functions to use along widgets
from IPython.display import clear_output, Image, display, Markdown, HTML
# widget packages
import ipywidgets as widgets
# https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e

OPEN_CV VER:	 3.2.0
NUMPY VER:	 1.16.2
TESSERECT VER:	 0.3.9


In [2]:
no_img = False
batt_out = False
no_img_count = 0
stop_looping = False

loop_delay = 60
adjusted_loop_delay = 0

## Camera Object setup

In [3]:
try:
    camera.close()
    print("Camera was open!")
    print("So just closed it")
except:
    print("Camera wasn't open ...")
    
    
    
def setup_cam_properties():
    print("Setting camera properties for image taking ...")
    camera.resolution = (320, 240)
    camera.framerate = 30
    # Set ISO to the desired value
    camera.iso = 200 
    # Valid values are between 0 (auto) and 1600. 
    # The actual value used when iso is explicitly set will be one 
    # of the following values(whichever is closest): 100, 200, 320, 400, 500, 640, 800
    # Wait for the automatic gain control to settle
    sleep(2)
    # Now fix the values
    camera.shutter_speed = camera.exposure_speed
    camera.exposure_mode = 'off'
    g = camera.awb_gains
    camera.awb_mode = 'off'
    camera.awb_gains = g
    
    print("\nCamera properties:")
    print("--------------------")
    print("RES:", camera.resolution)
    print("FR:", camera.framerate)
    print("ISO:", camera.iso)
    print("SS:", camera.shutter_speed)
    print("EXP_MODE:", camera.exposure_mode)
    print("AWB_GAIN:", camera.awb_gains)
    print("--------------------")

    
    
try:
    print("Opening Camera ...")
    camera = PiCamera()
except:
    print("Err Opening Camera ...")
    print("Closing first ...")
    camera.close()
    sleep(0.5)
    camera = PiCamera()
finally:
    print("Camera Opened!")
    setup_cam_properties()

Camera wasn't open ...
Opening Camera ...
Camera Opened!
Setting camera properties for image taking ...

Camera properties:
--------------------
RES: 320x240
FR: 30
ISO: 200
SS: 33164
EXP_MODE: off
AWB_GAIN: (Fraction(347, 256), Fraction(185, 128))
--------------------


# Capture and Pre Processing method

In [4]:
show_image_plots = False

In [5]:
def capture_image():
    global blurred
    
    # Capture and show raw camera content.
    # Also will be later used for openCV
    print("Capturing in RAW for OCR ...")
    rawCapture = PiRGBArray(camera, size=(320, 240))
    camera.capture(rawCapture, format="rgb")
    img = rawCapture.array
    
    print("Converting to RGB ...")
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print("Converting to GRAY ...")
    gray_img = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    # blurred = cv2.GaussianBlur(cropped_gray_img, (5, 5), 0)
    # blurred = cv2.GaussianBlur(gray_img, (5, 5), 0)
    print("Blurring to denoise ...")
    blurred = cv2.bilateralFilter(gray_img, 11, 17, 17) #Blur to reduce noise
    cropped_gray_img = blurred[20:180, 70:200] # [y:h, x:w]
    print("Writing file to disk for now ...")
    cv2.imwrite("processed_img.png", blurred)
    
    if show_image_plots:
        plt.imshow(img)
        plt.show()
        plt.imshow(rgb)
        plt.show()
        plt.imshow(gray_img, cmap="gray") # without thje cmap="gray" matplotlib doesn;t render gray.
        plt.show()
        plt.imshow(blurred, cmap="gray") # without thje cmap="gray" matplotlib doesn;t render gray.
        plt.show()
        
    # show small thumbnail image
    scale_percent = 50 # percent of original size
    width = int(cropped_gray_img.shape[1] * scale_percent / 100)
    height = int(cropped_gray_img.shape[0] * scale_percent / 100)
    dim = (width, height)
    smaller_img = cv2.resize(cropped_gray_img, dim, interpolation = cv2.INTER_AREA)
    def cm_to_inch(value):
        return value/2.54
    plt.figure(figsize=(cm_to_inch(6/2), cm_to_inch(7/2)))
    plt.imshow(smaller_img, cmap="gray")
    plt.show()
   
# capture_image()

# Google Cloud vision API based method to do OCR

[Guide here](https://pyimagesearch.com/2022/03/31/text-detection-and-ocr-with-google-cloud-vision-api/)

In [6]:
# create the client interface to access the Google Cloud Vision API
credentials = service_account.Credentials.from_service_account_file(filename="ocrtest-348215-181db3e230e2.json", scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = vision.ImageAnnotatorClient(credentials=credentials)

In [7]:
def getGoogleCloudOCRRes():
    global no_img
    global batt_out
    global no_img_count
    
    print("Loading file from disk to transfer to Google Vision API ...")
    with io.open("processed_img.png", "rb") as f:
        byteImage = f.read()

    # create an image object from the binary file and then make a request
    # to the Google Cloud Vision API to OCR the input image
    print("Making request to Google Cloud Vision API ...")
    image = vision.Image(content=byteImage)
    response = client.text_detection(image=image)
    
    # print(response.text_annotations)
    print("length of response: ", len(response.text_annotations))

    # check to see if there was an error when making a request to the API
    if response.error.message:
        raise Exception(
            "{}\nFor more info on errors, check:\n"
            "https://cloud.google.com/apis/design/errors".format(
                response.error.message))
    
    if len(response.text_annotations) is 0:
        no_img = True
        no_img_count = no_img_count +1
    else:
        no_img = False
        batt_out = False
        no_img_count = 0
        
        print("Current Date time:", dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        # loop over the Google Cloud Vision API OCR result response
        print("Found text/digits:")
        sleep(1)
        ocr_time = []
        for text in response.text_annotations[1::]:
            ocr = text.description
            ocr_time.append(ocr)
        if len(response.text_annotations) > 1:
            ocr_time.insert(1, ':')   
        print(''.join(ocr_time))

# Click function definition with Servo

In [8]:
servo_kit = ServoKit(channels=16, address=0x40, frequency=60) 
servo_kit.servo[0].actuation_range = 180

min_pos = 9
max_pos = 30
swing_delay = 0.5

start = 0 # for time keeping, to measure elapsed time per run, till OCR is finished

def click_with_servo(_):
    global start
    
    servo_kit.servo[0].angle = max_pos
    sleep(swing_delay)
    servo_kit.servo[0].angle = min_pos
    # Starts a timer to keep elapsed time for the process that will begin,
    # once the servo has pressed the button
    start = timeit.default_timer()
    # 
    sleep(swing_delay)
    servo_kit.servo[0].angle = max_pos    
    sleep(0.25)
    servo_kit.servo[0].angle = None

In [9]:
def validate_no_img_reason():
    global no_img
    global batt_out
    global no_img_count
    global stop_looping
    
    
    if no_img and no_img_count == 1:
        # no image found, will run again
        print("No image found, will try 2 more run cycles")
    elif no_img and no_img_count == 2:
        # no image found, will run again
        print("No image found, will try 1 more run cycle")
    elif no_img and no_img_count == 3:
        # battery dead (or something seriously wrong with google ocr)
        batt_out = True
        stop_looping = True
        
        print("No image found")
        sleep(1)
        print("Battery is probably dead")
        sleep(1)
        print("Stop Clicking the watch!")
        # dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    elif no_img == False:
        # OCR found, battery still have juice  
        print("Battery still has some juice")
    
        
        

def run_all_methods(_):
    global start
    global loop_delay
    global adjusted_loop_delay
    
    click_with_servo(_)
    sleep(0.5)
    capture_image()
    getGoogleCloudOCRRes()
    
    # Closes off the timer & calculate how much we need to adjust the delay between loops
    end = timeit.default_timer()
    elapsed_time = round(end-start, 2)
    adjusted_loop_delay = round(loop_delay-elapsed_time, 2)
    
    print("\nElapsed time in the process so far, since last button press: ", elapsed_time)
    print("For a loop_delay of", loop_delay, "secs, next process should begin in", adjusted_loop_delay, "secs")
    
    global no_img
    print('\nimage?', not no_img)
    
    validate_no_img_reason()

# Servo click button

In [10]:
click_btn = widgets.Button(
    description='click the watch',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='clicks the watch button with servo',
    icon='hand-o-left'
)


click_btn.on_click(run_all_methods)

# Pure Ugly loop method

In [11]:
count = 0

def begin_test_cycle(_):
    while True:
        global count
        global loop_delay
        global adjusted_loop_delay
        global stop_looping

        # Vars imported for restting, when battery is out
        global no_img
        global batt_out
        global no_img_count
        # -----------------------------------------------

        if stop_looping:
            no_img = False
            batt_out = False
            no_img_count = 0
            stop_looping = False
            break

        print("\nCLICK:", count)
        print("-------------")

        run_all_methods(_)
        count = count + 1
        sleep(adjusted_loop_delay)
        

# def stop_test_cycle(_):
#     global stop_looping
#     stop_looping = True
#     print ("Stopping next run ...")

# Loop Start/Stop Buttons

In [12]:
loop_begin_btn = widgets.Button(
    description='Begin Test Cycle',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='loops the test cycle',
    icon='refresh'
)

loop_begin_btn.on_click(begin_test_cycle)

# Display control buttons

In [13]:
display(click_btn)
display(loop_begin_btn)

Button(button_style='success', description='click the watch', icon='hand-o-left', style=ButtonStyle(), tooltip…

Button(button_style='warning', description='Begin Test Cycle', icon='refresh', style=ButtonStyle(), tooltip='l…